In [1]:
from os.path import abspath
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession \
    .builder \
    .appName("Hands On example") \
    .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to user050
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/22 12:53:25 WARN Utils: Service 'SparkUI' could not bind on port 20049. Attempting port 20050.
24/01/22 12:53:25 WARN Utils: Service 'SparkUI' could not bind on port 20050. Attempting port 20051.
24/01/22 12:53:25 WARN Utils: Service 'SparkUI' could not bind on port 20051. Attempting port 20052.
24/01/22 12:53:25 WARN Utils: Service 'SparkUI' could not bind on port 20052. Attempting port 20053.
24/01/22 12:53:25 WARN Utils: Service 'SparkUI' could not bind on port 20053. Attempting port 20054.
24/01/22 12:53:29 WARN HiveServer2CredentialProvider: Failed to get HS2 delegation token
java.util.NoSuchElementException: spark.sql.hive.hiveserver2.jdbc.url
	at org.apache.spark.SparkConf.$anonfun$get$1(SparkConf.scala:2

In [3]:
flights=spark.read.options(header='True',inferSchema='True',delimiter=',')\
    .csv("files/521972167_T_ONTIME.csv")
flights.show()

24/01/22 12:54:06 WARN ExecutorPodsSnapshotsStoreImpl: Exception when notifying snapshot subscriber.
io.fabric8.kubernetes.client.KubernetesClientException: Failure executing: POST at: https://172.20.0.1/api/v1/namespaces/mlx-user-2/pods. Message: pods "cdsw-g0iall5p0jvpi0ts-exec-1" already exists. Received status: Status(apiVersion=v1, code=409, details=StatusDetails(causes=[], group=null, kind=pods, name=cdsw-g0iall5p0jvpi0ts-exec-1, retryAfterSeconds=null, uid=null, additionalProperties={}), kind=Status, message=pods "cdsw-g0iall5p0jvpi0ts-exec-1" already exists, metadata=ListMeta(_continue=null, remainingItemCount=null, resourceVersion=null, selfLink=null, additionalProperties={}), reason=AlreadyExists, status=Failure, additionalProperties={}).
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.requestFailure(OperationSupport.java:681)
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.assertResponseCode(OperationSupport.java:620)
	at io.fabric8.kubernetes.client.ds

+----+-------+-----+------------+-----------+----------+--------------+----------+-------+--------+------+-----------------+---------------------+---------------------+------+----------------+----------------+-----------------+---------------+----------+---------------+-------------------+-------------------+----+--------------+--------------+---------------+-------------+--------+--------+---------+---------+-----------------+-------------+-------------+---------+--------------+-------------------+----+
|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|   FL_DATE|UNIQUE_CARRIER|AIRLINE_ID|CARRIER|TAIL_NUM|FL_NUM|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|ORIGIN_STATE_ABR|ORIGIN_STATE_FIPS|ORIGIN_STATE_NM|ORIGIN_WAC|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME|DEST_STATE_ABR|DEST_STATE_FIPS|DEST_STATE_NM|DEST_WAC|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_A

In [4]:
flights.schema

StructType(List(StructField(YEAR,IntegerType,true),StructField(QUARTER,IntegerType,true),StructField(MONTH,IntegerType,true),StructField(DAY_OF_MONTH,IntegerType,true),StructField(DAY_OF_WEEK,IntegerType,true),StructField(FL_DATE,StringType,true),StructField(UNIQUE_CARRIER,StringType,true),StructField(AIRLINE_ID,IntegerType,true),StructField(CARRIER,StringType,true),StructField(TAIL_NUM,StringType,true),StructField(FL_NUM,IntegerType,true),StructField(ORIGIN_AIRPORT_ID,IntegerType,true),StructField(ORIGIN_AIRPORT_SEQ_ID,IntegerType,true),StructField(ORIGIN_CITY_MARKET_ID,IntegerType,true),StructField(ORIGIN,StringType,true),StructField(ORIGIN_CITY_NAME,StringType,true),StructField(ORIGIN_STATE_ABR,StringType,true),StructField(ORIGIN_STATE_FIPS,IntegerType,true),StructField(ORIGIN_STATE_NM,StringType,true),StructField(ORIGIN_WAC,IntegerType,true),StructField(DEST_AIRPORT_ID,IntegerType,true),StructField(DEST_AIRPORT_SEQ_ID,IntegerType,true),StructField(DEST_CITY_MARKET_ID,IntegerType,tr

In [5]:
import pyspark.sql.functions as F
top10airports = flights\
    .where(flights["DAY_OF_WEEK"] >5)\
    .groupBy(["origin"])\
    .agg(F.count(F.lit(1)).alias("total_departures"))\
    .orderBy(F.desc("total_departures"))\
    .limit(10)\
    .show()


+------+----------------+
|origin|total_departures|
+------+----------------+
|   ATL|           10012|
|   ORD|            6579|
|   DEN|            6396|
|   LAX|            6026|
|   DFW|            5268|
|   SFO|            4601|
|   PHX|            4320|
|   SEA|            4105|
|   LAS|            4072|
|   MCO|            3682|
+------+----------------+

